In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

2024-02-06 19:49:30.034098: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 19:49:30.059734: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 19:49:30.059767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 19:49:30.059786: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 19:49:30.065313: I tensorflow/core/platform/cpu_feature_g

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


2024-02-06 19:49:31.448606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:49:31.452497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:49:31.452524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:49:31.457025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:49:31.457054: I tensorflow/compile

In [3]:
df = pd.read_csv('../../../datos/july_23/2023-07-01.csv')

In [4]:
(df["from_address"] == df["to_address"]).sum()

2108

In [5]:
df = df[df["from_address"] != df["to_address"]]

In [6]:
(df["from_address"] == df["to_address"]).sum()

0

In [7]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [8]:
len(set(df['from_address']).union(set(df['to_address'])))

310475

In [9]:
df = df[~df.to_address.isna()]
df = df[~df.from_address.isna()]

In [10]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [11]:
ids

{'0x5f112865c5134f1ac06e7a37644b5fc9239ebc81': 0,
 '0x3b8c2feb0f4953870f825df64322ec967aa26b8c': 1,
 '0x8c979c8dd0edf326dacac29c55c65f3f8d70b3f1': 2,
 '0xbf24def75d022f0049463c4a7f5484b223678269': 3,
 '0x6a706ebe9700a96b4954c4030345d40ffd67f28a': 4,
 '0xded79fd1d6d1a7badc10dce947e2cd32c6739d89': 5,
 '0xf8462ec42eeaa9e51bfbeed7903d92d9c44538cc': 6,
 '0x669653bb1501781d2514ab2cd315f2e83b33bf81': 7,
 '0xb13364ce9af538984911f1147f4148b46d24e5b7': 8,
 '0xc3cc3bd7dde6d26632465488be2344f5f5264f3f': 9,
 '0x50882c8b8aa5952f764fe000a4f794456217ddf1': 10,
 '0x379a2fc7f511636ecf8315ea79cf6fedbbecf7d1': 11,
 '0xaff78ca9fdbcc4f6b1bb5e76cf5143f5138f2cf9': 12,
 '0x31c6dcfbc18e262241c0924347b27286b74e9d13': 13,
 '0xa6804d66d1df16f2a9497ecf6546cd145f0b22e7': 14,
 '0xcbd3df9be71d9555449fbc051f32c1fb4e0446a1': 15,
 '0x8b9f5f3e9b15d0cc21d8874b21e50cfc4ed2e8bb': 16,
 '0x0c9cade04a40b3bcd4b522c9f958169c1e30af4f': 17,
 '0x7e910c5654fe0291b63990c73af20ef35f50591f': 18,
 '0x2d05d53055658b566ae80a305092125fe4852

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            39740288  
                                                                 
 dense (Dense)               (None, 1, 64)             8256      
                                                                 
 batch_normalization (Batch  (None, 1, 64)             256       
 Normalization)                                                  
                                                                 
Total params: 39748800 (151.63 MB)
Trainable params: 39748672 (151.63 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [13]:
# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [14]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        init = index * self.batch_size
        end = (index + 1) * self.batch_size

        batch = self.df[init:end]
        negative = self.df.sample(len(batch))

        anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
        positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
        negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
        
        anchor = tf.convert_to_tensor(anchor)
        positive = tf.convert_to_tensor(positive)
        negative = tf.convert_to_tensor(negative)
        
        # el fake target simunla lo que sería aprendizaje supervisado        
        fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

        return ([anchor, positive, negative], [fake_target]*3)

In [15]:
generator = GeneratorTriplet(df, ids, 64)

In [16]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=0.001, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)

        # distance_positive = tf.norm(anchor - positive, axis=-1)
        # distance_negative = tf.norm(anchor - negative, axis=-1)
        distance_positive = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        distance_negative = tf.reduce_sum(tf.square(anchor - negative), axis=1)

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)
        return tf.reduce_mean(loss)

loss = TripletCustom()    

model.compile(
    optimizer=Adam(0.00001),
    loss=loss
)

In [17]:
model.fit(generator, epochs=2)

Epoch 1/2


2024-02-06 19:49:36.945748: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f6f85d27f00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-06 19:49:36.945787: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-02-06 19:49:36.949560: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-06 19:49:37.071543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-06 19:49:37.126051: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4005/4005 [==============================] - 76s 18ms/step - loss: 0.4249
Epoch 2/2
4005/4005 [==============================] - 62s 15ms/step - loss: 0.2803


In [18]:
# input_layer = Input(shape=(2,))
# dense_layer = Dense(units=1, activation='linear')(input_layer)
# model = Model(inputs=input_layer, outputs=dense_layer)
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.summary()
# y_train = np.random.rand(100)
# print(model.fit(combined_tensor, y_train, epochs=1, batch_size=32))